# API general information

API key:
- Your API key is 1953847a1db7b2070fcd10ee09add91c
- Within the next couple of hours, it will be activated and ready to use
- You can later create more API keys on your account page
- Please, always use your API key in each API call

Endpoint:
- Please, use the endpoint api.openweathermap.org for your API calls
- Example of API call:
api.openweathermap.org/data/2.5/weather?q=London,uk&APPID=1953847a1db7b2070fcd10ee09add91c

Useful links:
- API documentation https://openweathermap.org/api
- Details of your plan https://openweathermap.org/price
- Please, note that 16-days daily forecast and History API are not available for Free subscribers


In [2]:
import requests
import pandas as pd
import numpy as np
import json
import pprint
import time
import string

In [2]:
pd_location = pd.read_excel('population_CH_2021.xlsx')
df = pd_location[1:]
df.head()

,Municipality,Population on 31 December
1,- Zürich,1564662
2,>> Bezirk Affoltern,56370
3,......0001 Aeugst am Albis,1988
4,......0002 Affoltern am Albis,12404
5,......0003 Bonstetten,5632


In [3]:
# throwing away the strange characters
df['Municipality'] = df['Municipality'].str.split(' ', n=1).str[1]

<ipython-input-3-78093223db43>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Municipality'] = df['Municipality'].str.split(' ', n=1).str[1]


In [4]:
print(df['Municipality'])

1                   Zürich
2         Bezirk Affoltern
3          Aeugst am Albis
4       Affoltern am Albis
5               Bonstetten
               ...        
2331          Vendlincourt
2332         Basse-Allaine
2333         Clos du Doubs
2334           Haute-Ajoie
2335            La Baroche
Name: Municipality, Length: 2335, dtype: object


In [5]:
# Renaming columns
df = df.rename(columns={'Population on 31 December':'Population_End2021'})
print(df.shape)
df.head(50)

(2335, 2)


,Municipality,Population_End2021
1,Zürich,1564662
2,Bezirk Affoltern,56370
3,Aeugst am Albis,1988
4,Affoltern am Albis,12404
5,Bonstetten,5632
6,Hausen am Albis,3852
7,Hedingen,3780
8,Kappel am Albis,1290
9,Knonau,2445
10,Maschwanden,641


In [6]:
# Use case-insensitive search to create a mask of rows containing the word "Berzirk" and remove these rows
mask = df['Municipality'].str.contains('Bezirk', case=False)
df = df.drop(df[mask].index)

# Use case-insensitive search to create a mask of rows containing the word "District" and remove these rows
mask2 = df['Municipality'].str.contains('District', case=False)
df = df.drop(df[mask2].index)

# Remove the rows containing the word "Berzirk" using the mask and the drop() method

df.reset_index(drop=True)

df.head(50)

,Municipality,Population_End2021
1,Zürich,1564662
3,Aeugst am Albis,1988
4,Affoltern am Albis,12404
5,Bonstetten,5632
6,Hausen am Albis,3852
7,Hedingen,3780
8,Kappel am Albis,1290
9,Knonau,2445
10,Maschwanden,641
11,Mettmenstetten,5570


In [ ]:
#  Zürich is the largest city in Switzerland with a population of over 428'700
# still missing to delete "(ZH)" and others

In [7]:
list_cities = list(df['Municipality'])
print(len(list_cities))
list_cities

2244


['Zürich',
 'Aeugst am Albis',
 'Affoltern am Albis',
 'Bonstetten',
 'Hausen am Albis',
 'Hedingen',
 'Kappel am Albis',
 'Knonau',
 'Maschwanden',
 'Mettmenstetten',
 'Obfelden',
 'Ottenbach',
 'Rifferswil',
 'Stallikon',
 'Wettswil am Albis',
 'Adlikon',
 'Benken (ZH)',
 'Berg am Irchel',
 'Buch am Irchel',
 'Dachsen',
 'Dorf',
 'Feuerthalen',
 'Flaach',
 'Flurlingen',
 'Andelfingen',
 'Henggart',
 'Humlikon',
 'Kleinandelfingen',
 'Laufen-Uhwiesen',
 'Marthalen',
 'Ossingen',
 'Rheinau',
 'Thalheim an der Thur',
 'Trüllikon',
 'Truttikon',
 'Volken',
 'Stammheim',
 'Bachenbülach',
 'Bassersdorf',
 'Bülach',
 'Dietlikon',
 'Eglisau',
 'Embrach',
 'Freienstein-Teufen',
 'Glattfelden',
 'Hochfelden',
 'Höri',
 'Hüntwangen',
 'Kloten',
 'Lufingen',
 'Nürensdorf',
 'Oberembrach',
 'Opfikon',
 'Rafz',
 'Rorbas',
 'Wallisellen',
 'Wasterkingen',
 'Wil (ZH)',
 'Winkel',
 'Bachs',
 'Boppelsen',
 'Buchs (ZH)',
 'Dällikon',
 'Dänikon',
 'Dielsdorf',
 'Hüttikon',
 'Neerach',
 'Niederglatt',
 '

In [17]:
# Mobility output
mobility_example = {
    
    "geometry": {"x": 8.463, "y": 47.245, "spatialReference": {"wkid": 4326}},
  
    "attributes": {
      "provider.id": "string",
      "provider.name": "string",
      "provider.timezone": "string",
      "provider.apps.ios.store_uri": "string",
      "provider.apps.android.store_uri": "string",
      "id": "string",
      "pickup_type": "free_floating",
      "station.name": "string",
      "station.address": "string",
      "station.postcode": "string",
      "station.status.installed": 'true',
      "station.status.renting": 'true',
      "station.status.returning": 'true',
      "station.status.num_vehicle_available": 0,
      "station.status.num_vehicle_disabled": 0,
      "vehicle.status.disabled": 'true',
      "vehicle.status.reserved": 'true',
      "vehicle_type": "string"
    },
    "featureId": "string",
    "id": "string",
    "layerBodId": "string",
    "layerName": "string"
  }


In [ ]:
# Output example from mobility data
coordinates_mobility = {"geometry":{
    "x": 0, 
    "y": 0, 
    "spatialReference": {"wkid": 4326}}

In [20]:
def extract_coordinates(mobility):
    
    values = {
        'provider_id': mobility["attributes"]["provider.id"],
        'provider_name': mobility["attributes"]["provider.name"],
        'timezone': mobility["attributes"]["provider.timezone"],
        'geometry': {mobility['geometry']["x"], mobility['geometry']["y"]}
    }
            
    return values
    

In [22]:
# test extraction for one mobility provider
mobility_list = extract_coordinates(mobility_example)
mobility_list

{'provider_id': 'string',
 'provider_name': 'string',
 'timezone': 'string',
 'geometry': {8.463, 47.245}}

In [ ]:
# Download all country records
list_coordinates = []
for coord in mobility_list:
    values = extract_values_ct(country)
    list_countries.append(values)
    

# Data quality check
print(len(list_countries))
list_countries[:3]

In [8]:
# Replace with the city and country code you want to get the weather for
#city_names = ["Zermatt", "Andermatt", "Visp", "Oberwald", "Disentis/Mustér"]
API_KEY = "f929f8bae1cf9df1f10a9a2a99661e67"
location = list_cities.copy()
country_code = "CH"
units = "metric"
LAT = 47.245   # 'y' in cartesiane system
LON = 8.463

In [ ]:
# Create empyt list for current weather
current_weather = []
current_coordinates = mobility_list

for city_name in location:
    # Base URL for OpenWeather API
    #url = f"http://api.openweathermap.org/data/2.5/weather?q={location},{country_code}&units={units}&appid={API_KEY}"
    url = f"https://api.openweathermap.org/data/2.5/weather?lat={LAT}&lon={LON}&&units={units}&appid={API_KEY}"
       
# Fetch the current weather data
    response = requests.get(url)
    data = json.loads(response.text)
    current_weather.append(data)


In [9]:
current_weather[0]

NameError: name 'current_weather' is not defined

# Mobility Coordinates

In [11]:
# Fetch mobility data
API = "https://sharedmobility.ch/station_information.json"
response = requests.get(API)
data = response.json()
data = data['data']['stations']

In [12]:
data

[{'station_id': 'edrivecarsharing:566',
  'name': 'edrive Renault Zoe TG 197 872',
  'lat': 47.514965,
  'lon': 8.93666,
  'provider_id': 'edrivecarsharing'},
 {'station_id': 'edrivecarsharing:638',
  'name': 'edrive Renault Zoe TG 197 606',
  'lat': 47.515022,
  'lon': 8.936639,
  'provider_id': 'edrivecarsharing'},
 {'station_id': 'edrivecarsharing:776',
  'name': 'Martishöhe Renault Zoé',
  'lat': 47.13029,
  'lon': 8.197523,
  'provider_id': 'edrivecarsharing'},
 {'station_id': 'edrivecarsharing:808',
  'name': 'LANDI Thun Renault Zoe',
  'lat': 46.77251,
  'lon': 7.63081,
  'provider_id': 'edrivecarsharing'},
 {'station_id': 'edrivecarsharing:839',
  'name': 'LANDI Thun Renault Zoe',
  'lat': 46.8159,
  'lon': 7.5097,
  'provider_id': 'edrivecarsharing'},
 {'station_id': 'edrivecarsharing:906',
  'name': 'LANDI Aare Nissan Leaf',
  'lat': 46.88048,
  'lon': 7.614559,
  'provider_id': 'edrivecarsharing'},
 {'station_id': 'edrivecarsharing:935',
  'name': 'Reserve Willisau Renault Z

In [13]:
data = data[:100]
len(data)

100

In [14]:
# API parameters
API_KEY = "f929f8bae1cf9df1f10a9a2a99661e67"
units = "metric"
current_weather = []

# Fetch station's  weather data
for i in range(len(data)):
    LAT = data[i]['lat']
    LON = data[i]['lon']
    url = f"https://api.openweathermap.org/data/2.5/weather?lat={LAT}&lon={LON}&&units={units}&appid={API_KEY}"
    response = requests.get(url)
    r = json.loads(response.text)
    current_weather.append(r)
    #time.sleep(1)
    
    

In [15]:
print(type(current_weather))
current_weather

<class 'list'>


[{'coord': {'lon': 8.9367, 'lat': 47.515},
  'weather': [{'id': 800,
    'main': 'Clear',
    'description': 'clear sky',
    'icon': '01d'}],
  'base': 'stations',
  'main': {'temp': 3.11,
   'feels_like': 3.11,
   'temp_min': -0.28,
   'temp_max': 5.96,
   'pressure': 971,
   'humidity': 68},
  'visibility': 10000,
  'wind': {'speed': 0.46, 'deg': 235, 'gust': 0.91},
  'clouds': {'all': 10},
  'dt': 1680764490,
  'sys': {'type': 2,
   'id': 2081038,
   'country': 'CH',
   'sunrise': 1680756827,
   'sunset': 1680803963},
  'timezone': 7200,
  'id': 2659724,
  'name': 'Matzingen',
  'cod': 200},
 {'coord': {'lon': 8.9366, 'lat': 47.515},
  'weather': [{'id': 800,
    'main': 'Clear',
    'description': 'clear sky',
    'icon': '01d'}],
  'base': 'stations',
  'main': {'temp': 3.11,
   'feels_like': 3.11,
   'temp_min': -0.28,
   'temp_max': 5.96,
   'pressure': 971,
   'humidity': 68},
  'visibility': 10000,
  'wind': {'speed': 0.46, 'deg': 235, 'gust': 0.91},
  'clouds': {'all': 10},


In [16]:
# Define function to extract weather values from each mobility station
def extract_weather_values(location): 
    try:        
        sea_level = location['main']['sea_level']
    except KeyError:
        sea_level = None
        
    try:        
        ground_level = location['main']['grnd_level']
    except KeyError:
        ground_level = None
        
    try:        
        wind_gust = location['wind']['gust']
    except KeyError:
        wind_gust = None
    
    values = {
        'id_weather': location['sys']['id'],
        'weather_main': location['weather'][0]['main'],
        'weather_description': location['weather'][0]['description'],
        'temp': location['main']['temp'],
        'temp_fells': location['main']['feels_like'],
        'temp_min': location['main']['temp_min'],
        'temp_max': location['main']['temp_max'],
        'pressure': location['main']['pressure'],
        'humidity': location['main']['humidity'],
        'sea_level': sea_level,
        'ground_level': ground_level,
        'visibility': location['visibility'],       
        'wind_speed': location['wind']['speed'],
        'wind_degrees': location['wind']['deg'],
        'wind_gust': wind_gust,
        'datetime': location['dt'],
        'country_code': location['sys']['country'],
        'sunrise': location['sys']['sunrise'],
        'sunset': location['sys']['sunset'],
        'timezone': location['timezone'],
        'location': location['name'],      
    }
            
    return values

In [ ]:
# getting weather variables
for i, item in enumerate(list_location):       
    id_weather= item.get('id_weather', None)
    weather_main= iitem.get('weather_main', None)
    weather_description= iitem.get('weather_description', None)
    temp= item.get('temp', None)
    temp_fells= item.get('temp_fells', None)
    temp_min= item.get('temp_min', None)
    temp_max= item.get('temp_max', None)
    pressure= item.get('pressure', None)
    humidity= item.get('humidity', None)
    sea_level= item.get('sea_level', None)
    ground_level= item.get('ground_level', None)
    visibility= item.get('visibility', None)   
    wind_speed= litem.get('wind_speed', None)
    wind_degrees= item.get('wind_degrees', None)
    wind_gust= item.get('wind_gust', None)
    datetime= litem.get('datetime', None)
    country_code= item.get('country_code', None)
    sunrise= item.get('sunrise', None)
    sunset= item.get('sunset', None)
    timezone= item.get('timezone', None)
    location= item.get('location', None)    
   
            
  

In [17]:
# Extract values for all coordinate locations
list_location = []
for i, weather_data in enumerate(current_weather):
    values = extract_weather_values(weather_data)
    list_location.append(values)
    
print(len(list_location))
list_location

100


[{'id_weather': 2081038,
  'weather_main': 'Clear',
  'weather_description': 'clear sky',
  'temp': 3.11,
  'temp_fells': 3.11,
  'temp_min': -0.28,
  'temp_max': 5.96,
  'pressure': 971,
  'humidity': 68,
  'sea_level': None,
  'ground_level': None,
  'visibility': 10000,
  'wind_speed': 0.46,
  'wind_degrees': 235,
  'wind_gust': 0.91,
  'datetime': 1680764490,
  'country_code': 'CH',
  'sunrise': 1680756827,
  'sunset': 1680803963,
  'timezone': 7200,
  'location': 'Matzingen'},
 {'id_weather': 2081038,
  'weather_main': 'Clear',
  'weather_description': 'clear sky',
  'temp': 3.11,
  'temp_fells': 3.11,
  'temp_min': -0.28,
  'temp_max': 5.96,
  'pressure': 971,
  'humidity': 68,
  'sea_level': None,
  'ground_level': None,
  'visibility': 10000,
  'wind_speed': 0.46,
  'wind_degrees': 235,
  'wind_gust': 0.91,
  'datetime': 1680764490,
  'country_code': 'CH',
  'sunrise': 1680756827,
  'sunset': 1680803963,
  'timezone': 7200,
  'location': 'Matzingen'},
 {'id_weather': 6933,
  'w

In [18]:
df = pd.DataFrame(list_location)
df

,id_weather,weather_main,weather_description,temp,temp_fells,temp_min,temp_max,pressure,humidity,sea_level,...,visibility,wind_speed,wind_degrees,wind_gust,datetime,country_code,sunrise,sunset,timezone,location
0,2081038,Clear,clear sky,3.11,3.11,-0.28,5.96,971,68,NaN,...,10000,0.46,235,0.91,1680764490,CH,1680756827,1680803963,7200,Matzingen
1,2081038,Clear,clear sky,3.11,3.11,-0.28,5.96,971,68,NaN,...,10000,0.46,235,0.91,1680764490,CH,1680756827,1680803963,7200,Matzingen
2,6933,Clear,clear sky,1.83,1.83,-0.70,5.04,1019,72,NaN,...,10000,0.45,195,0.89,1680764491,CH,1680757029,1680804116,7200,Sempach
3,2000117,Clear,clear sky,2.69,1.26,-0.98,5.99,1020,57,NaN,...,10000,1.54,150,NaN,1680764491,CH,1680757187,1680804230,7200,Steffisburg
4,2000117,Clear,clear sky,2.88,1.47,-1.91,5.95,1020,56,NaN,...,10000,1.54,150,NaN,1680764491,CH,1680757214,1680804262,7200,Mühlethurnen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2012960,Clear,clear sky,2.89,1.48,-0.73,6.21,1021,64,NaN,...,10000,1.54,150,NaN,1680764510,CH,1680757226,1680804290,7200,Berne
96,2019255,Clear,clear sky,3.09,3.09,0.07,6.16,1021,71,NaN,...,10000,0.51,0,NaN,1680764510,CH,1680756935,1680804053,7200,Zürich (Kreis 4) / Werd
97,6940,Clear,clear sky,2.90,2.90,-0.66,5.98,1019,78,NaN,...,10000,0.89,174,2.24,1680764486,CH,1680757216,1680804312,7200,Grenchen
98,6937,Clear,clear sky,2.55,1.10,-0.67,5.71,1020,65,NaN,...,10000,1.54,150,NaN,1680764122,CH,1680757222,1680804286,7200,Bern


In [22]:
df_fields = df.transpose()
df_fields = df_fields.iloc[:,:1] 
df_fields

,0
id_weather,2081038
weather_main,Clear
weather_description,clear sky
temp,3.11
temp_fells,3.11
temp_min,-0.28
temp_max,5.96
pressure,971
humidity,68
sea_level,NaN


In [19]:
df.columns

Index(['id_weather', 'weather_main', 'weather_description', 'temp',
       'temp_fells', 'temp_min', 'temp_max', 'pressure', 'humidity',
       'sea_level', 'ground_level', 'visibility', 'wind_speed', 'wind_degrees',
       'wind_gust', 'datetime', 'country_code', 'sunrise', 'sunset',
       'timezone', 'location'],
      dtype='object')

## Filtering by big cities

This may be needed later on to move the data to the warehouse

In [233]:
# Selecting cities greater than 10K inhabitants
big_cities = df.loc[df['Population_End2021'] > 10000]
big_cities.reset_index(drop=True, inplace=True)
big_cities

,Municipality,Population_End2021
0,Zürich,1564662
1,Affoltern am Albis,12404
2,Bassersdorf,11960
3,Bülach,22193
4,Kloten,20779
...,...,...
234,Vernier,35491
235,Versoix,13280
236,Veyrier,11837
237,Jura,73798


In [199]:
big_cities.drop(columns="index")

,Municipality,Population_End2021
0,Zürich,1564662
1,Bezirk Affoltern,56370
2,Affoltern am Albis,12404
3,Bezirk Andelfingen,32136
4,Bezirk Bülach,158566
...,...,...
317,Jura,73798
318,District de Delémont,39125
319,Delémont,12479
320,District des Franches-Montagnes,10478


In [15]:
df['Municipality'] = df['Municipality'].str.split('Bezirk ').str[1]
df

<ipython-input-15-5d4e38206811>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Municipality'] = df['Municipality'].str.split('Bezirk ').str[1]


,Municipality,Population on 31 December
1,NaN,1564662
2,Affoltern,56370
3,NaN,1988
4,NaN,12404
5,NaN,5632
...,...,...
2331,NaN,559
2332,NaN,1225
2333,NaN,1284
2334,NaN,1073


In [187]:
# Find the index of the duplicated rows
duplicates = big_cities.duplicated(keep=False)
duplicate_indexes = big_cities.loc[duplicates].index

Int64Index([], dtype='int64')

In [182]:
# make list of row indices where 'gmail.com' is in any column
indices = big_cities['Municipality'].where(big_cities['Municipality'].apply(lambda x: x.str.contains('Bezirk'))).dropna(how='all').index.tolist()
indices

AttributeError: 'str' object has no attribute 'str'

In [184]:
print(len(big_cities))
big_cities

322


,index,Municipality,Population_End2021
0,1,Zürich,1564662
1,2,Bezirk Affoltern,56370
2,4,Affoltern am Albis,12404
3,17,Bezirk Andelfingen,32136
4,40,Bezirk Bülach,158566
...,...,...,...
317,2279,Jura,73798
318,2280,District de Delémont,39125
319,2288,Delémont,12479
320,2300,District des Franches-Montagnes,10478
